In [ ]:
import os
import csv
import h5py
import random
import cv2 as cv
import numpy as np
import pandas as pd

from google.colab import drive
from time import gmtime, strftime
from google.colab.patches import cv2_imshow as cv_imshow

In [ ]:
drive.mount('/content/drive')  

Mounted at /content/drive


In [ ]:
!cd '/content/drive/MyDrive/Colab Notebooks/Computer Vision/Project/dataset/' && ls -la

total 1504266
-rw-------  1 root root  167085842 Jan 13 06:52 dataset_256.h5
-rw-------  1 root root 1368448332 Jan 13 06:29 dataset.h5
drwx------ 69 root root       4096 Dec 24 06:43 pre_processed_images_256
drwx------  2 root root       4096 Dec 23 18:52 pre_processed_images_512
drwx------  2 root root       4096 Jan  8 08:19 pre_processed_images_64
drwx------  2 root root       4096 Dec 22 00:10 pre_processed_silhouettes
-rw-------  1 root root        168 Mar 12  2021 sample_submission.csv
drwx------  2 root root       4096 Dec 22 17:55 test_images
-rw-------  1 root root    4807443 Mar 12  2021 train.csv
-rw-------  1 root root        151 Jan  8 08:00 train.gsheet
drwx------  2 root root       4096 Dec 22 17:55 train_images


#### Constants

In [1]:
# Desired image size
IMG_SIZE = 128 

# Include a random silhouette mask in the images
MASK_IMG = False

In [ ]:
train_imgs_dir = '/content/drive/MyDrive/Colab Notebooks/Computer Vision/Project/dataset/train_images'
pre_processed_imgs_dir = '/content/drive/MyDrive/Colab Notebooks/Computer Vision/Project/dataset/pre_processed_images_256'
pre_processed_silhouettes_dir = '/content/drive/MyDrive/Colab Notebooks/Computer Vision/Project/dataset/pre_processed_silhouettes'

train_filepath = '/content/drive/MyDrive/Colab Notebooks/Computer Vision/Project/dataset/train.csv'
hdf5_dataset_file = '/content/drive/MyDrive/Colab Notebooks/Computer Vision/Project/dataset/dataset_128.h5'

## Silhouette masks

**Note:** This requires running the setup scripts for the silhouettes dataset prior!

In [ ]:
!mkdir '/content/drive/MyDrive/Colab Notebooks/Computer Vision/Project/dataset/pre_processed_silhouettes/'

In [ ]:
silhouettes_imgs_dir = '/content/drive/MyDrive/Colab Notebooks/Computer Vision/Project/dataset/silhouettes'
pre_processed_silhouettes_imgs_dir = '/content/drive/MyDrive/Colab Notebooks/Computer Vision/Project/dataset/pre_processed_silhouettes/'
silhouette_positions = ['lying', 'sitting', 'standing']
silhouette_sizes = [128, 256, 512]

def process_silhouette(position, tag, size, id):
  silhouette = cv.imread(f'{silhouettes_imgs_dir}/{silhouette_positions[position]}/{tag}.jpg', cv.IMREAD_GRAYSCALE)

  padding = int((IMG_SIZE - size) / 2)
  silhouette = cv.resize(silhouette, (size, size))
  silhouette = cv.copyMakeBorder(silhouette, padding, padding, padding, padding, cv.BORDER_CONSTANT, None, [255, 255, 255])

  cv.imwrite(f'{pre_processed_silhouettes_imgs_dir}/{id}.jpg', silhouette)

In [ ]:
id = 1

for position in range(0, 3):
  for tag in range(0, 1200):
    for size in silhouette_sizes:
      process_silhouette(position, tag + 1, size, id)

      id += 1

print(id)

In [ ]:
!rm -fr silhouettes-of-human-posture.zip
!rm -fr silhouettes

## Pre-Process Images

In [ ]:
def square_resize_img(img):
  height, width, _ = img.shape

  min_dim = min(height, width)

  y_delta = int((height-min_dim)/2)
  x_delta = int((width-min_dim)/2)

  img = img[y_delta:y_delta + min_dim, x_delta:x_delta + min_dim]
  img = cv.resize(img, (IMG_SIZE, IMG_SIZE))

  return img

def mask_img(img):
  mask_id = random.randint(0, 10800)
  silhouette = cv.imread(f'{pre_processed_silhouettes_dir}/{mask_id}.jpg', cv.IMREAD_GRAYSCALE)
  silhouette = cv.threshold(silhouette, 128, 255, cv.THRESH_BINARY)[1]
  
  img = cv.bitwise_and(img, img, mask = silhouette)

  return img

def pre_process_img(chain_folder, img_id):
  img = cv.imread(f'{train_imgs_dir}/{chain_folder}/{img_id}')

  if img is None:
    return

  img = square_resize_img(img)

  if MASK_IMG:
    img = mask_img(img)

  if not os.path.isdir(f'{pre_processed_imgs_dir}/{chain_folder}'):
    os.mkdir(f'{pre_processed_imgs_dir}/{chain_folder}')

  cv.imwrite(f'{pre_processed_imgs_dir}/{chain_folder}/{img_id}', img)

In [ ]:
!mkdir '/content/drive/MyDrive/Colab Notebooks/Computer Vision/Project/dataset/pre_processed_images_128/'

In [ ]:
train_csv = pd.read_csv(train_filepath, parse_dates = ["timestamp"])

Remove duplicate images.

In [ ]:
train_csv = train_csv.drop_duplicates(subset = ["image"], keep = "first")

Pre-process every image and save them in a folder.

In [ ]:
last_index = 0

for index, row in train_csv.iterrows():
  if not os.path.isfile(f'{pre_processed_imgs_dir}/{row["chain"]}/{row["image"]}'):
    last_index = index
    break

print(last_index)

In [ ]:
print(f'DATE          TIME         INDEX')

for index, row in train_csv.iterrows():
  if index < last_index:
    continue

  # Print every 1000 images for sanity check
  if index % 1000 == 0:
    print(f'{strftime("%d/%m/%Y    %H:%M:%S")}     {index}')

  pre_process_img(row['chain'], row['image'])


## Create HDF5 Dataset

In [ ]:
def convert_img_2_hdf5(hf, path, name):
  img = cv.imread(path)

  img = cv.resize(img, (IMG_SIZE, IMG_SIZE))

  if name in hf:
    del hf[name]

  hf.create_dataset(
   name = name,
   data = img,
   shape = (IMG_SIZE, IMG_SIZE, 3),
   compression = "gzip",
   compression_opts = 9
  )

In [ ]:
hf = h5py.File(hdf5_dataset_file, 'r+')

Read csv file.

In [ ]:
train_csv = pd.read_csv(train_filepath, parse_dates=["timestamp"])

Remove duplicate images.

In [ ]:
train_csv = train_csv.drop_duplicates(subset = ["image"], keep = "first")

Add image path (absolute path) attribute to facilitate reading when training.

In [ ]:
train_csv['image_path'] = pre_processed_imgs_dir + '/' + train_csv['chain'].astype(str) + '/' + train_csv['image']

Convert every image to the HDF5 dataset.

In [ ]:
last_index = 0

for index, row in train_csv.iterrows():  
  if row['image'] not in hf:
    last_index = index
    break

print(last_index)

In [ ]:
print(f'DATE          TIME         INDEX')

for index, row in train_csv.iterrows():
  if index < last_index:
    continue

  if index % 1000 == 0:
    print(f'{strftime("%d/%m/%Y    %H:%M:%S")}     {index}')

  convert_img_2_hdf5(hf, row['image_path'], row['image'])

Check images that were incorrectly converted and re-convert them.

In [ ]:
for index, row in train_csv.iterrows():  
  if not hf[row['image']].dtype.type == np.uint8:
    print(index)
    convert_img_2_hdf5(hf, row['image_path'], row['image'])

Close file to guarantee data is saved.

In [ ]:
hf.close()